In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
import torchvision
from torchvision import models
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import os

import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, FastICA
from sklearn.covariance import LedoitWolf, MinCovDet

from torchvision.transforms import transforms
from sklearn.metrics import roc_auc_score

np.random.seed(252525)
torch.manual_seed(252525)

import torch
import torch.nn as nn

from meanshift import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:

net = models.efficientnet_b4(pretrained=True).features
net = net.to(device)
net = net.eval()
device

'cuda'

In [38]:
bs = 50
clazz = 3
epochs = 1
augment = False

log = []

for run in range(2):

    for clazz in range(15):

        X_, X_valid_, X_test_, X_labels_, T = zip(*[dataloader(clazz, P=224, s=224, label_per_patch=False, augment=augment) for i in range(epochs)])
        X_, X_valid_, X_test_ = np.concatenate(X_), np.concatenate(X_valid_), np.concatenate(X_test_)

        scores_inliers = []
        scores_outliers = []

        for l in range(9):

            X = concat_features(X_, net, layers=[l], fmap_pool=False)
            X_valid = concat_features(X_valid_, net, layers=[l], fmap_pool=False)
            X_test = concat_features(X_test_, net, layers=[l], fmap_pool=False)

            # Pooling training
            n = X.shape[0]
            X__ = X.mean((2,3))
            #X__ = X__.reshape(n, -1)

            C_inv = LedoitWolf().fit(X__).precision_
            mu = X__.mean(0)

            # Pooling inliers
            n_valid = len(X_valid)
            X_valid__ = X_valid.mean((2,3)) - mu
            #X_valid_ = X_valid_.reshape(n_valid, -1)

            # Pooling outliers
            n_test = len(X_test)
            X_test__ = X_test.mean((2,3)) - mu
            #X_test_ = X_test_.reshape(n_test, -1)

            # Mahalanobis
            M_valid_ = np.sqrt( ( X_valid__ * ( C_inv @ X_valid__.T ).T ).sum(1) )    
            M_test_  = np.sqrt( ( X_test__ * ( C_inv @ X_test__.T ).T ).sum(1) )    

            scores_inliers.append(M_valid_)
            scores_outliers.append(M_test_)

        auc = roc_auc_score([0] * n_valid + [1] * n_test, np.concatenate([np.asarray(scores_inliers).sum(0), np.asarray(scores_outliers).sum(0)]))

        log.append(pd.DataFrame(np.asarray([run, clazz, l, n, auc])[:, None].T, columns=["run", "class", "layers", "n_samples", "auc"])) 

        print(MVTEC.CLASSES[clazz], auc)

bottle 0.9976190476190476
carpet 0.9333868378812198
leather 1.0
pill 0.9015275504637207
tile 0.9873737373737375
wood 0.9956140350877192
cable 0.9439655172413794
grid 0.9039264828738512
toothbrush 0.9805555555555556
zipper 0.983718487394958
capsule 0.9234144395692063
hazelnut 0.9917857142857143
metal_nut 0.9281524926686217
screw 0.7200245952039355
transistor 0.9616666666666667
bottle 0.9976190476190476
carpet 0.9333868378812198
leather 1.0
pill 0.9015275504637207
tile 0.9873737373737375
wood 0.9956140350877192
cable 0.9439655172413794
grid 0.9039264828738512
toothbrush 0.9805555555555556
zipper 0.983718487394958
capsule 0.9234144395692063


KeyboardInterrupt: 

In [39]:
log_ = pd.concat(log)

for clazz in range(15):
    auc = log_[ (log_["class"] == clazz) ].groupby("run").mean()["auc"]
    n = int(log_[ (log_["class"] == clazz) ].groupby("run").mean()["n_samples"].mean())
    print(f"class={MVTEC.CLASSES[clazz]:10s}, n_samples={n:3d}", f"\tauc: {auc.mean():.3f}", "\u00B1", f"{auc.std():.3f}" )

avg = log_[ (log_["class"] == clazz) ].groupby("run").mean()["auc"].mean()
print(avg)

class=bottle    , n_samples=209 	auc: 0.998 ± 0.000
class=carpet    , n_samples=280 	auc: 0.933 ± 0.000
class=leather   , n_samples=245 	auc: 1.000 ± 0.000
class=pill      , n_samples=267 	auc: 0.902 ± 0.000
class=tile      , n_samples=230 	auc: 0.987 ± 0.000
class=wood      , n_samples=247 	auc: 0.996 ± 0.000
class=cable     , n_samples=224 	auc: 0.944 ± 0.000
class=grid      , n_samples=264 	auc: 0.904 ± 0.000
class=toothbrush, n_samples= 60 	auc: 0.981 ± 0.000
class=zipper    , n_samples=240 	auc: 0.984 ± 0.000
class=capsule   , n_samples=219 	auc: 0.923 ± 0.000
class=hazelnut  , n_samples=391 	auc: 0.992 ± nan
class=metal_nut , n_samples=220 	auc: 0.928 ± nan
class=screw     , n_samples=320 	auc: 0.720 ± nan
class=transistor, n_samples=213 	auc: 0.962 ± nan
0.9616666666666667


In [38]:
bs = 50
epochs = 1

log = []

for n in [10, 25, 50, 80, 100]: #1, 5, 
    for run in range(2):

        for clazz in range(15):

            X_, X_valid_, X_test_, X_labels_, T = zip(*[dataloader(clazz, P=224, s=224, label_per_patch=False, augment=augment) for i in range(epochs)])
            X_, X_valid_, X_test_ = np.concatenate(X_), np.concatenate(X_valid_), np.concatenate(X_test_)
            X_ = X_[np.random.permutation(len(X_))[:n]]

            scores_inliers = []
            scores_outliers = []

            for l in range(9):

                X = concat_features(X_, net, blocks=[l], fmap_pool=False)
                X_valid = concat_features(X_valid_, net, blocks=[l], fmap_pool=False)
                X_test = concat_features(X_test_, net, blocks=[l], fmap_pool=False)

                # Pooling training
                m = X.shape[0]
                X__ = X.mean((2,3))
                #X__ = X__.reshape(n, -1)
                
                try:
                    C_inv = LedoitWolf().fit(X__).precision_
                    mu = X__.mean(0)

                    # Pooling inliers
                    n_valid = len(X_valid)
                    X_valid__ = X_valid.mean((2,3)) - mu
                    #X_valid_ = X_valid_.reshape(n_valid, -1)

                    # Pooling outliers
                    n_test = len(X_test)
                    X_test__ = X_test.mean((2,3)) - mu
                    #X_test_ = X_test_.reshape(n_test, -1)

                    # Mahalanobis
                    M_valid_ = np.sqrt( ( X_valid__ * ( C_inv @ X_valid__.T ).T ).sum(1) )    
                    M_test_  = np.sqrt( ( X_test__ * ( C_inv @ X_test__.T ).T ).sum(1) )    

                    scores_inliers.append(M_valid_)
                    scores_outliers.append(M_test_)
                except:
                    print(f"Failure in class:{clazz}, n:{n}, l:{l}")

            auc = roc_auc_score([0] * n_valid + [1] * n_test, np.concatenate([np.asarray(scores_inliers).sum(0), np.asarray(scores_outliers).sum(0)]))

            log.append(pd.DataFrame(np.asarray([run, clazz, l, n, auc])[:, None].T, columns=["run", "class", "layers", "n_samples", "auc"])) 

            print(MVTEC.CLASSES[clazz], auc)

Failure in class:0, n:10, l:0
bottle 0.984920634920635
Failure in class:1, n:10, l:0
carpet 0.812199036918138
leather 0.9972826086956521
pill 0.73240589198036
tile 0.9794372294372294
wood 0.9868421052631579
cable 0.8290854572713644
grid 0.6482873851294904
toothbrush 0.8944444444444445
zipper 0.9642857142857142
Failure in class:10, n:10, l:0
capsule 0.7483047467092142
hazelnut 0.8385714285714285
metal_nut 0.8470185728250244
screw 0.5763476122156179
Failure in class:14, n:10, l:0
transistor 0.8529166666666667
bottle 0.9880952380952381
carpet 0.9542536115569823
leather 0.9952445652173914
Failure in class:3, n:10, l:0
pill 0.7452264048008729
tile 0.9823232323232324
wood 0.9850877192982456
cable 0.8615067466266866
grid 0.6942355889724311
toothbrush 0.95
zipper 0.9569327731092436
capsule 0.8607897885919427
hazelnut 0.9010714285714286
metal_nut 0.8543499511241447
screw 0.46320967411354785
transistor 0.8687499999999999
bottle 0.9952380952380953
carpet 0.9482343499197431
leather 0.9986413043478

In [42]:
dat = pd.concat(log)

x = dat[["run", "class", "auc", "n_samples"]].astype(np.float32).groupby("n_samples").mean().index
y = dat[["run", "class", "auc", "n_samples"]].astype(np.float32).groupby("n_samples").mean()["auc"]
s = dat[["run", "class", "auc", "n_samples"]].astype(np.float32).groupby(["run", "n_samples"]).mean().groupby([ "n_samples"]).std()["auc"]

np.asarray(x), np.asarray(y), np.asarray(s)

(array([  1.,   5.,  10.,  25.,  50.,  80., 100.]),
 array([0.5       , 0.831542  , 0.87037206, 0.89370525, 0.9181281 ,
        0.9368697 , 0.9367348 ], dtype=float32),
 array([0.        , 0.00013533, 0.02060096, 0.01077063, 0.00744486,
        0.00024028, 0.00020155]))